In [1]:
import PIL
import gc
import torch
import torchvision
import os
import sys

import numpy as np
import matplotlib.pyplot as plt  
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import Subset
from IPython.core.display import display, HTML
from numpy.random import RandomState
from wide_resnet import WideResNet
from auto_augment import AutoAugment, Cutout
from efficientnet_pytorch import EfficientNet
from cifar_loader import SmallSampleController

sys.path.insert(0,'glico-learning-small-sample/glico_model')

from tester import runGlico



# display(HTML("<style>.container { width:40% !important; }</style>"))


In [2]:

def getAcc(preds,targets):
    return np.sum([1 if preds[i] == targets[i] else 0 for i in range(len(preds))])/len(preds)

def train(model, device, train_loader, optimizer, epoch, display=True):
    """
    Summary: Implements the training procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    optimizer: the optimizer for our training
    train_loader: dataloader for our train data
    display: output flag
    == output ==
    the mean train loss, the train accuracy
    """
    
    lossTracker = []
    
    targets=[]
    preds=[]
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        lossTracker.append(loss.detach())
        with torch.no_grad():
            pred = torch.argmax(output,1).cpu().numpy()
            preds.extend(pred)
            targets.extend(target.cpu().numpy())
        
    lossTracker = [x.item() for x in lossTracker]
    meanLoss = np.mean(lossTracker)
    accuracy = getAcc(preds,targets)
    if display:
        print('Train Epoch: {} [acc: {:.0f}%]\tLoss: {:.6f}'.format(
          epoch, 100. * accuracy, meanLoss))
        
    return accuracy, meanLoss


def glicoTrain(model, device, train_loader, optimizer, epoch, glicoLoader,replaceProb=0.5,display=True):
    """
    Summary: Implements the training procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    optimizer: the optimizer for our training
    train_loader: dataloader for our train data
    display: output flag
    == output ==
    the mean train loss, the train accuracy
    """
    
    lossTracker = []
    
    targets=[]
    preds=[]
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #replace samples with samples from glico with probability replaceprob
        data = glicoLoader.replaceBatch(data,target,replaceProb) 
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        lossTracker.append(loss.detach())
        with torch.no_grad():
            pred = torch.argmax(output,1).cpu().numpy()
            preds.extend(pred)
            targets.extend(target.cpu().numpy())
        
    lossTracker = [x.item() for x in lossTracker]
    meanLoss = np.mean(lossTracker)
    accuracy = getAcc(preds,targets)
    if display:
        print('Train Epoch: {} [acc: {:.0f}%]\tLoss: {:.6f}'.format(
          epoch, 100. * accuracy, meanLoss))
        
    return accuracy, meanLoss



def test(model, device, test_loader,verbose=True):
    """
    Summary: Implements the testing procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    test_loader: dataloader for our test data
    verbose: output flag
    == output ==
    the mean test loss, the test accuracy
    """
    
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    meanLoss = test_loss / len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    if verbose: print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        mean_test_loss, correct, len(test_loader.dataset),
        accuracy))
        
    return accuracy, meanLoss


def checkTest(model,device,valSets,valTracker,latexTracker,epoch,
              model_name,optim_name,lr,totalTestSamples,seed,verbose=True):
    """
    Summary: checks the test accuracy, prints, and saves statistics
    """
    tempAcc = []
    tempLoss = []
    for val_loader in valSets:
        acc,loss = test(model, device, val_loader,verbose = False)
        tempAcc.append(acc)
        tempLoss.append(loss)
        
    meanAcc = np.mean(tempAcc)
    stdAcc = np.std(tempAcc)
    
    meanLoss = np.mean(tempLoss)
    if verbose:
        print('[Trained for {} epochs and tested on {} sets of 2000 images]\
        Avg Acc: {:.2f} +- {:.2f} , Avg Loss: {:.2f}'.format(
            epoch,VALIDATION_SET_NUM,meanAcc,stdAcc,meanLoss))
        
        
    tableRow = getLatexRow(architecture=model_name,epoch=epoch,accuracy=meanAcc,optim=optim_name,
                           lr=lr,totalTestSamples=totalTestSamples,dataAug="Nothing",
                           seed=seed,title=False)
    
    latexTracker.append(tableRow)
        
    valTracker["allLoss"].extend(tempLoss)
    valTracker["allAcc"].extend(tempAcc)
    valTracker["meanLoss"].append(meanLoss)
    valTracker["meanAcc"].append(meanAcc)
    valTracker["stdAcc"].append(stdAcc)





In [3]:
def getLatexRow(architecture,epoch,accuracy,optim,lr,
                totalTestSamples,dataAug,seed,title=False):
    """
    Summary: generates one row of latex for a results table
    """
    categories = ["Model","Epoch","Accuracy","Optimizer","lr","Test Sample Num",
                  "data augmentation","seed"]
    row = [str(architecture),str(epoch),str(round(accuracy,3)),str(optim),
           str(lr),str(totalTestSamples),str(dataAug),str(seed)]
    
    if title:
        c = "&".join(categories)
        r = "&".join(row)
        return "{}\\\\\n{}\\\\".format(c,r)
    else:
        r = "&".join(row)
        return "{}\\\\".format(r)
    
    
def plot(xlist,ylist,xlab,ylab,title,color,label,savedir=".",save=False):
    """
    Summary: plots the given list of numbers against its idices and 
    allows for high resolution saving
    """
    fig = plt.figure()
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.plot(xlist,ylist,color=color,marker=".",label=label)
    plt.legend()
    
    if save:
        if not os.path.isdir(savedir):
            os.mkdir(savedir)
        filepath = os.path.join(savedir,"{}".format(title))
        plt.savefig(filepath+".pdf")
        os.system("pdftoppm -png -r 300 {}.pdf {}.png".format(filepath,filepath))
        
    plt.show()
    
    

In [4]:
def getModel(model_name):
    if "wide" in model_name.lower():
        return WideResNet(28, 10, num_classes=10)
    elif "fix" in model_name.lower():
        return EfficientNet.from_pretrained(model_name) # change to not be pretrained
    
    
def getOptimizer128(optimizer_name,parameters):
    if "sgd" in  optimizer_name.lower():
        LR = 0.09
        optim = torch.optim.SGD(parameters, 
                                  lr=LR, momentum=0.9,
                                  weight_decay=0.0005)
        return optim, LR
    elif "adam" in optimizer_name.lower():
        LR = 0.001
        optim = torch.optim.Adam(parameters, 
                              lr=LR, weight_decay=0)
        return optim, LR
        
    

In [5]:
torch.cuda.empty_cache()
gc.collect()

OPTIM = "Adam"
MODEL = "WideResNet28"
EPOCH_NUM = 2000
TRAIN_SAMPLE_NUM = 100
VAL_SAMPLE_NUM = 2000
BATCH_SIZE = 128
VALIDATION_SET_NUM = 1
AUGMENT = True
VAL_DISPLAY_DIVISOR = 25
CIFAR_TRAIN = True
REPLACE_PROB = 0.05
SEED = None

#cifar-10:
#mean = (0.4914, 0.4822, 0.4465)
#std = (0.247, 0.243, 0.261)

normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                  std=[0.247, 0.243, 0.261])

# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                   std=[0.229, 0.224, 0.225])
if AUGMENT:
    dataAugmentation = [ 
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        AutoAugment(),
        Cutout()
    ]
    augment = "Crop,Flip,AutoAugment,Cutout"
else: 
    dataAugmentation = []
    augment = "Nothing"



transform_train = transforms.Compose(dataAugmentation + [transforms.ToTensor(), normalize]) 
transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same

glico_train = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, download=True)
cifar_train = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_train, download=True)
cifar_val = datasets.CIFAR10(root='.',train=CIFAR_TRAIN, transform=transform_val, download=True)

ss = SmallSampleController(numClasses=10,trainSampleNum=TRAIN_SAMPLE_NUM, # abstract the data-loading procedure
                           valSampleNum=VAL_SAMPLE_NUM, batchSize=BATCH_SIZE, 
                           multiplier=VALIDATION_SET_NUM, trainDataset=cifar_train, 
                           valDataset=cifar_val)
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data, valSets, seed = ss.generateNewSet(device,valMultiplier = VALIDATION_SET_NUM) #Sample from datasets


train_labeled_dataset = Subset(glico_train, ss.trainSampler.indexes[0]) #get the same subset without transform
nagTrainer = runGlico(train_labeled_dataset=train_labeled_dataset, classes=10,epochs=750)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Generated new permutation of the CIFAR train dataset with                 seed:1620010424, train sample num: 100, test sample num: 2000



[*** NEW PRINT ***]nag_params: NAGParams(nz=512, force_l2=False, is_pixel=True, z_init='rndm', is_classifier=True, disc_net='conv', loss='ce', data_name='cifar-10', noise_proj=True, shot=0), rn: tester.py_cifar100_test_run, dataset <class 'list'>



[DEBUG] data name=cifar-10, data res = 32
[NEW CODE] image_params:ImageParams(sz=(32, 32), nc=3, n=100, mu=None, sd=None)
not eval | augment : TRUE
=>Generated data loader, res=32, workers=6 transform=Compose(
    ToTensor()
) sampler=None
[DEBUG] dataset name: <class 'torch.utils.data.dataset.subset'>,offset_labels=0
torch.Size([100, 512])
init_models_weights(self): init rndm
num classes = 10
dataset size = 100
not eval | augment : TRUE
=>Generated data loader, res=32, workers=6 transform=Co

/home/benjamin/Documents/github/COMP499DL/src/glico-learning-small-sample/glico_model/utils.py:168: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(x)


Epoch: [0][0/1]	Time  0.645 ( 0.645)	Data  0.260 ( 0.260)	Loss 8.55879 (8.55879)	d_Loss 2.32977 (2.32977)
=>NAG Epoch: 0 Error: 8.558786392211914, Time:     0.01m
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netZ_nag_0.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netG_nag_0.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netD_nag_0.pkl
Show images...
=>epoch:1/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [1][0/1]	Time  0.300 ( 0.300)	Data  0.271 ( 0.271)	Loss 8.51396 (8.51396)	d_Loss 2.34086 (2.34086)
=>NAG Epoch: 1 Error: 8.513964653015137, Time:     0.01m
=>epoch:2/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [2][0/1]	Time  0.296 ( 0.296)	Data  0.267 ( 0.267)	Loss 8.42989 (8.42989)	d_Loss 2.35310 (2.35310)
=>NAG Epoch: 2 Error: 8.429893493652344, Time:     0.01m
=>epoch:3/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [3][0/1]	Time  0.294 ( 0.294)	Data  0.264 ( 0.264)	Loss 8.27003 (8.27003)	d_Loss 2.31783 (2.31783)
=>N

Epoch: [31][0/1]	Time  0.296 ( 0.296)	Data  0.266 ( 0.266)	Loss 7.07978 (7.07978)	d_Loss 1.89031 (1.89031)
=>NAG Epoch: 31 Error: 7.079784393310547, Time:     0.01m
=>epoch:32/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [32][0/1]	Time  0.293 ( 0.293)	Data  0.264 ( 0.264)	Loss 7.08693 (7.08693)	d_Loss 1.87127 (1.87127)
=>NAG Epoch: 32 Error: 7.086932182312012, Time:     0.01m
=>consecutive_loss: 1
=>epoch:33/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [33][0/1]	Time  0.295 ( 0.295)	Data  0.264 ( 0.264)	Loss 7.02147 (7.02147)	d_Loss 1.86036 (1.86036)
=>NAG Epoch: 33 Error: 7.021470546722412, Time:     0.01m
=>epoch:34/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [34][0/1]	Time  0.310 ( 0.310)	Data  0.280 ( 0.280)	Loss 7.02162 (7.02162)	d_Loss 1.83507 (1.83507)
=>NAG Epoch: 34 Error: 7.021623134613037, Time:     0.01m
=>consecutive_loss: 1
=>epoch:35/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [35][0/1]	Time  0.289 ( 0.289)	Data  0.259 ( 0.259)	Loss 6.95557 

Epoch: [63][0/1]	Time  0.292 ( 0.292)	Data  0.262 ( 0.262)	Loss 6.15942 (6.15942)	d_Loss 1.65037 (1.65037)
=>NAG Epoch: 63 Error: 6.159422874450684, Time:     0.01m
=>epoch:64/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [64][0/1]	Time  0.294 ( 0.294)	Data  0.262 ( 0.262)	Loss 6.13697 (6.13697)	d_Loss 1.62957 (1.62957)
=>NAG Epoch: 64 Error: 6.136966705322266, Time:     0.01m
=>epoch:65/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [65][0/1]	Time  0.302 ( 0.302)	Data  0.271 ( 0.271)	Loss 6.08325 (6.08325)	d_Loss 1.61898 (1.61898)
=>NAG Epoch: 65 Error: 6.083247184753418, Time:     0.01m
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netZ_nag_65.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netG_nag_65.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netD_nag_65.pkl
=>epoch:66/750  resolution:32 batch:128, lr1.000000e-03
Epoch: [66][0/1]	Time  0.307 ( 0.307)	Data  0.275 ( 0.275)	Loss 6.08051 (6.08051)	d_Loss 1.60427 (1.60427)
=>NAG

...

Epoch: [592][0/1]	Time  0.291 ( 0.291)	Data  0.262 ( 0.262)	Loss 3.60584 (3.60584)	d_Loss 1.46277 (1.46277)
=>NAG Epoch: 592 Error: 3.6058402061462402, Time:     0.01m
=>consecutive_loss: 3
=>epoch:593/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [593][0/1]	Time  0.290 ( 0.290)	Data  0.261 ( 0.261)	Loss 3.60599 (3.60599)	d_Loss 1.46284 (1.46284)
=>NAG Epoch: 593 Error: 3.605985403060913, Time:     0.01m
=>consecutive_loss: 4
=>epoch:594/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [594][0/1]	Time  0.288 ( 0.288)	Data  0.259 ( 0.259)	Loss 3.60406 (3.60406)	d_Loss 1.46287 (1.46287)
=>NAG Epoch: 594 Error: 3.604062795639038, Time:     0.01m
=>epoch:595/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [595][0/1]	Time  0.300 ( 0.300)	Data  0.270 ( 0.270)	Loss 3.60455 (3.60455)	d_Loss 1.46281 (1.46281)
=>NAG Epoch: 595 Error: 3.6045451164245605, Time:     0.01m
=>consecutive_loss: 1
=>epoch:596/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [596][0/1]	Time  0.298 ( 0.29

Epoch: [626][0/1]	Time  0.300 ( 0.300)	Data  0.270 ( 0.270)	Loss 3.59499 (3.59499)	d_Loss 1.46268 (1.46268)
=>NAG Epoch: 626 Error: 3.5949912071228027, Time:     0.01m
=>epoch:627/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [627][0/1]	Time  0.299 ( 0.299)	Data  0.268 ( 0.268)	Loss 3.59770 (3.59770)	d_Loss 1.46261 (1.46261)
=>NAG Epoch: 627 Error: 3.5977025032043457, Time:     0.01m
=>consecutive_loss: 1
=>epoch:628/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [628][0/1]	Time  0.301 ( 0.301)	Data  0.271 ( 0.271)	Loss 3.59483 (3.59483)	d_Loss 1.46274 (1.46274)
=>NAG Epoch: 628 Error: 3.5948326587677, Time:     0.01m
=>epoch:629/750  resolution:32 batch:128, lr3.906250e-06
Epoch: [629][0/1]	Time  0.302 ( 0.302)	Data  0.270 ( 0.270)	Loss 3.59701 (3.59701)	d_Loss 1.46262 (1.46262)
=>NAG Epoch: 629 Error: 3.597005844116211, Time:     0.01m
=>consecutive_loss: 1
=>epoch:630/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [630][0/1]	Time  0.310 ( 0.310)	Data  0.280 ( 0.280)	

Epoch: [659][0/1]	Time  0.290 ( 0.290)	Data  0.261 ( 0.261)	Loss 3.59238 (3.59238)	d_Loss 1.46248 (1.46248)
=>NAG Epoch: 659 Error: 3.5923776626586914, Time:     0.01m
=>consecutive_loss: 10
=>epoch:660/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [660][0/1]	Time  0.306 ( 0.306)	Data  0.274 ( 0.274)	Loss 3.59066 (3.59066)	d_Loss 1.46248 (1.46248)
=>NAG Epoch: 660 Error: 3.5906574726104736, Time:     0.01m
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netZ_nag_660.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netG_nag_660.pkl
Save checkpoints...! runs/nets_tester.py_cifar100_test_run/netD_nag_660.pkl
=>epoch:661/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [661][0/1]	Time  0.298 ( 0.298)	Data  0.267 ( 0.267)	Loss 3.58993 (3.58993)	d_Loss 1.46253 (1.46253)
=>NAG Epoch: 661 Error: 3.5899322032928467, Time:     0.01m
=>epoch:662/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [662][0/1]	Time  0.313 ( 0.313)	Data  0.283 ( 0.283)	Loss 3.59125 (

Epoch: [690][0/1]	Time  0.290 ( 0.290)	Data  0.260 ( 0.260)	Loss 3.58800 (3.58800)	d_Loss 1.46243 (1.46243)
=>NAG Epoch: 690 Error: 3.5879971981048584, Time:     0.01m
=>consecutive_loss: 2
=>epoch:691/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [691][0/1]	Time  0.303 ( 0.303)	Data  0.273 ( 0.273)	Loss 3.58690 (3.58690)	d_Loss 1.46243 (1.46243)
=>NAG Epoch: 691 Error: 3.586901903152466, Time:     0.01m
=>consecutive_loss: 3
=>epoch:692/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [692][0/1]	Time  0.291 ( 0.291)	Data  0.261 ( 0.261)	Loss 3.58552 (3.58552)	d_Loss 1.46244 (1.46244)
=>NAG Epoch: 692 Error: 3.585519552230835, Time:     0.01m
=>epoch:693/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [693][0/1]	Time  0.289 ( 0.289)	Data  0.260 ( 0.260)	Loss 3.58700 (3.58700)	d_Loss 1.46235 (1.46235)
=>NAG Epoch: 693 Error: 3.58699893951416, Time:     0.01m
=>consecutive_loss: 1
=>epoch:694/750  resolution:32 batch:128, lr1.953125e-06
Epoch: [694][0/1]	Time  0.290 ( 0.290)

Epoch: [724][0/1]	Time  0.296 ( 0.296)	Data  0.266 ( 0.266)	Loss 3.58417 (3.58417)	d_Loss 1.46233 (1.46233)
=>NAG Epoch: 724 Error: 3.5841710567474365, Time:     0.01m
=>consecutive_loss: 10
=>epoch:725/750  resolution:32 batch:128, lr9.765625e-07
Epoch: [725][0/1]	Time  0.289 ( 0.289)	Data  0.260 ( 0.260)	Loss 3.58429 (3.58429)	d_Loss 1.46228 (1.46228)
=>NAG Epoch: 725 Error: 3.5842902660369873, Time:     0.01m
=>consecutive_loss: 11
=>epoch:726/750  resolution:32 batch:128, lr9.765625e-07
Epoch: [726][0/1]	Time  0.290 ( 0.290)	Data  0.260 ( 0.260)	Loss 3.58444 (3.58444)	d_Loss 1.46227 (1.46227)
=>NAG Epoch: 726 Error: 3.5844411849975586, Time:     0.01m
=>consecutive_loss: 12
=>epoch:727/750  resolution:32 batch:128, lr9.765625e-07
Epoch: [727][0/1]	Time  0.295 ( 0.295)	Data  0.265 ( 0.265)	Loss 3.58311 (3.58311)	d_Loss 1.46227 (1.46227)
=>NAG Epoch: 727 Error: 3.583106517791748, Time:     0.01m
=>consecutive_loss: 13
=>epoch:728/750  resolution:32 batch:128, lr9.765625e-07
Epoch: [7

In [6]:
import importlib
import interpolate
importlib.reload(interpolate)
import interpolate
from interpolate import interpolate_points


def getGenImage(z1,z2,gen,steps,codeSize):
    interp = interpolate_points(z1,z2,n_steps=steps, slerp=True, print_mode=False)
    code = torch.cuda.FloatTensor(steps, codeSize).normal_(0, 0.15)
    im = gen(interp,code)
    return im

In [7]:
import time
import itertools
class GlicoLoader:
    
    def getInterpolations(self):
        interps = [[x for x in itertools.combinations(self.netZ.label2idx[y], 2)] for y in range(10)]
        self.interps = {}
        for i,combinations in enumerate(interps):
            zvecs = self.netZ(torch.tensor(combinations).cuda())
            temp = []
            for idx in range(zvecs.size(0)):
                z1 = zvecs[idx,0,:]
                z2 = zvecs[idx,1,:]
                gen = getGenImage(z1,z2,self.netG,steps=self.steps,codeSize= self.nag.code_size)
                temp.append(gen.detach())
                
            self.interps[i] = torch.cat(temp,dim=0)
            
        
    def __init__(self,nagTrainer,steps):
        self.nagTrainer = nagTrainer
        self.nag = nagTrainer.nag
        self.netZ = self.nag.netZ
        self.netG = self.nag.netG
        self.steps = steps
        self.getInterpolations()
        
        self.indices = [0 for x in range(10)]
        a = self.interps[0].size(0)
        prng = RandomState(int(time.time()))
        self.indexers = [list(prng.permutation(np.arange(0,a))) for x in range(10)]
        
        
    def sample(self,classNum):
        if self.interps[0].size(0) == self.indices[classNum]:
            self.indices[classNum] = 0
            prng = RandomState(int(time.time()))
            self.indexers[classNum] = list(prng.permutation(np.arange(0,self.interps[0].size(0))))
            
        temp = self.interps[classNum][self.indexers[classNum][self.indices[classNum]],:,:,:]
            
        return temp
    
    def replaceBatch(self,batch,targets,replaceProb):
        for x in range(batch.size(0)):
            prob = np.random.rand()
            if prob < replaceProb:
                batch[x,:,:,:] = self.sample(targets[x].item())
            else:
                pass
        return batch
        
        
        
        

In [8]:
glicoLoader = GlicoLoader(nagTrainer,5)

In [9]:
import matplotlib

c_to_idx = {"airplane" : 0,
"automobile" : 1,
"bird" : 2,
"cat" : 3,
"deer" : 4,
"dog" : 5,
"frog" : 6,
"horse" : 7,
"ship" : 8,
"truck" : 9}

# idx_to_c =  {v:k for k,v in c_to_idx.items()}
# save = True
# matplotlib.use( 'agg' )

# for _ in range(5):
#     for class_ in range(10):
#         gen = glicoLoader.sample(classNum=class_)
#         plt.imshow(gen.cpu().permute(1, 2, 0))
#         plt.axis("off")
#         if save: 
#             plt.savefig(os.path.join("{}{}.pdf".format(idx_to_c[class_],_)),
#                 bbox_inches="tight",pad_inches=0)
        


# plt.show()

In [10]:

model = getModel(MODEL).cuda()
optimizer,LR = getOptimizer128(OPTIM,model.parameters())

print(' => Total trainable parameters: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))        

trainTracker = {"meanLoss":[],"accuracy":[]}
valTracker = {"allLoss":[],"allAcc":[],"meanLoss":[],"meanAcc":[],"stdAcc":[]}
latexTracker = []

print("Begin Train for {} epochs".format(EPOCH_NUM))
for epoch in range(EPOCH_NUM):
    acc, loss = glicoTrain(model, device, train_data[0], optimizer, 
                      epoch+1,glicoLoader,replaceProb=REPLACE_PROB, display=True)
    trainTracker["meanLoss"].append(loss)
    trainTracker["accuracy"].append(acc)
    
    if (epoch+1) % VAL_DISPLAY_DIVISOR == 0 or epoch == 0:
        checkTest(model,device,valSets,valTracker,latexTracker,epoch+1,
              model_name=MODEL,optim_name=OPTIM,lr=LR,totalTestSamples=VAL_SAMPLE_NUM*VALIDATION_SET_NUM,
                  seed=seed,verbose=True)
        
          
        
        

 => Total trainable parameters: 36.48M
Begin Train for 2000 epochs
Train Epoch: 1 [acc: 7%]	Loss: 2.310721
Train Epoch: 2 [acc: 13%]	Loss: 2.516140
Train Epoch: 3 [acc: 13%]	Loss: 2.499593
Train Epoch: 4 [acc: 13%]	Loss: 2.367805
Train Epoch: 5 [acc: 18%]	Loss: 2.383141
Train Epoch: 6 [acc: 17%]	Loss: 2.221243
Train Epoch: 7 [acc: 19%]	Loss: 2.216757
Train Epoch: 8 [acc: 24%]	Loss: 2.215537
Train Epoch: 9 [acc: 17%]	Loss: 2.174945
Train Epoch: 10 [acc: 15%]	Loss: 2.187501
Train Epoch: 11 [acc: 18%]	Loss: 2.144031
Train Epoch: 12 [acc: 23%]	Loss: 2.054556
Train Epoch: 13 [acc: 26%]	Loss: 2.151839
Train Epoch: 14 [acc: 17%]	Loss: 2.124151
Train Epoch: 15 [acc: 17%]	Loss: 2.195880
Train Epoch: 16 [acc: 24%]	Loss: 2.128168
Train Epoch: 17 [acc: 27%]	Loss: 2.124317
Train Epoch: 18 [acc: 30%]	Loss: 2.045443
Train Epoch: 19 [acc: 23%]	Loss: 2.133019
Train Epoch: 20 [acc: 27%]	Loss: 2.000544
Train Epoch: 21 [acc: 22%]	Loss: 1.996009
Train Epoch: 22 [acc: 25%]	Loss: 2.046394
Train Epoch: 23 [ac

/home/benjamin/venv/torch11/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


[Trained for 25 epochs and tested on 5 sets of 2000 images]        Avg Acc: 16.11 +- 0.43 , Avg Loss: 4.98
Train Epoch: 26 [acc: 20%]	Loss: 2.116291
Train Epoch: 27 [acc: 23%]	Loss: 2.069545
Train Epoch: 28 [acc: 23%]	Loss: 2.121457
Train Epoch: 29 [acc: 27%]	Loss: 2.002287
Train Epoch: 30 [acc: 19%]	Loss: 2.124004
Train Epoch: 31 [acc: 21%]	Loss: 2.069081
Train Epoch: 32 [acc: 24%]	Loss: 2.072709
Train Epoch: 33 [acc: 21%]	Loss: 2.114552
Train Epoch: 34 [acc: 19%]	Loss: 2.056277
Train Epoch: 35 [acc: 21%]	Loss: 2.025362
Train Epoch: 36 [acc: 22%]	Loss: 1.961313
Train Epoch: 37 [acc: 23%]	Loss: 2.024331
Train Epoch: 38 [acc: 23%]	Loss: 1.964233
Train Epoch: 39 [acc: 25%]	Loss: 2.103429
Train Epoch: 40 [acc: 35%]	Loss: 1.966812
Train Epoch: 41 [acc: 32%]	Loss: 1.965144
Train Epoch: 42 [acc: 24%]	Loss: 2.013800
Train Epoch: 43 [acc: 28%]	Loss: 1.985408
Train Epoch: 44 [acc: 29%]	Loss: 1.889034
Train Epoch: 45 [acc: 35%]	Loss: 1.959948
Train Epoch: 46 [acc: 31%]	Loss: 1.979236
Train Epoch

[Trained for 200 epochs and tested on 5 sets of 2000 images]        Avg Acc: 28.62 +- 0.91 , Avg Loss: 3.03
Train Epoch: 201 [acc: 54%]	Loss: 1.424509
Train Epoch: 202 [acc: 50%]	Loss: 1.508255
Train Epoch: 203 [acc: 44%]	Loss: 1.575523
Train Epoch: 204 [acc: 48%]	Loss: 1.422364
Train Epoch: 205 [acc: 48%]	Loss: 1.377148
Train Epoch: 206 [acc: 49%]	Loss: 1.414764
Train Epoch: 207 [acc: 40%]	Loss: 1.554871
Train Epoch: 208 [acc: 48%]	Loss: 1.445502
Train Epoch: 209 [acc: 52%]	Loss: 1.471260
Train Epoch: 210 [acc: 51%]	Loss: 1.464094
Train Epoch: 211 [acc: 52%]	Loss: 1.395939
Train Epoch: 212 [acc: 54%]	Loss: 1.533380
Train Epoch: 213 [acc: 53%]	Loss: 1.476205
Train Epoch: 214 [acc: 53%]	Loss: 1.431048
Train Epoch: 215 [acc: 47%]	Loss: 1.399814
Train Epoch: 216 [acc: 48%]	Loss: 1.541440
Train Epoch: 217 [acc: 44%]	Loss: 1.533271
Train Epoch: 218 [acc: 45%]	Loss: 1.582577
Train Epoch: 219 [acc: 47%]	Loss: 1.486198
Train Epoch: 220 [acc: 45%]	Loss: 1.440551
Train Epoch: 221 [acc: 47%]	Loss

Train Epoch: 374 [acc: 75%]	Loss: 0.707633
Train Epoch: 375 [acc: 77%]	Loss: 0.777782
[Trained for 375 epochs and tested on 5 sets of 2000 images]        Avg Acc: 28.88 +- 0.58 , Avg Loss: 3.39
Train Epoch: 376 [acc: 69%]	Loss: 0.877320
Train Epoch: 377 [acc: 75%]	Loss: 0.729101
Train Epoch: 378 [acc: 75%]	Loss: 0.773058
Train Epoch: 379 [acc: 71%]	Loss: 0.849803
Train Epoch: 380 [acc: 79%]	Loss: 0.662570
Train Epoch: 381 [acc: 76%]	Loss: 0.800932
Train Epoch: 382 [acc: 77%]	Loss: 0.716508
Train Epoch: 383 [acc: 72%]	Loss: 0.803547
Train Epoch: 384 [acc: 76%]	Loss: 0.758541
Train Epoch: 385 [acc: 81%]	Loss: 0.665457
Train Epoch: 386 [acc: 82%]	Loss: 0.667425
Train Epoch: 387 [acc: 72%]	Loss: 0.879336
Train Epoch: 388 [acc: 73%]	Loss: 0.869117
Train Epoch: 389 [acc: 78%]	Loss: 0.822597
Train Epoch: 390 [acc: 76%]	Loss: 0.775286
Train Epoch: 391 [acc: 71%]	Loss: 0.850131
Train Epoch: 392 [acc: 73%]	Loss: 0.819240
Train Epoch: 393 [acc: 77%]	Loss: 0.694973
Train Epoch: 394 [acc: 62%]	Loss

...

Train Epoch: 1234 [acc: 90%]	Loss: 0.302717
Train Epoch: 1235 [acc: 93%]	Loss: 0.175379
Train Epoch: 1236 [acc: 89%]	Loss: 0.335007
Train Epoch: 1237 [acc: 94%]	Loss: 0.208150
Train Epoch: 1238 [acc: 86%]	Loss: 0.472129
Train Epoch: 1239 [acc: 93%]	Loss: 0.247372
Train Epoch: 1240 [acc: 91%]	Loss: 0.211595
Train Epoch: 1241 [acc: 86%]	Loss: 0.348800
Train Epoch: 1242 [acc: 91%]	Loss: 0.190233
Train Epoch: 1243 [acc: 94%]	Loss: 0.198584
Train Epoch: 1244 [acc: 88%]	Loss: 0.359482
Train Epoch: 1245 [acc: 93%]	Loss: 0.210928
Train Epoch: 1246 [acc: 88%]	Loss: 0.297268
Train Epoch: 1247 [acc: 93%]	Loss: 0.238180
Train Epoch: 1248 [acc: 92%]	Loss: 0.289185
Train Epoch: 1249 [acc: 87%]	Loss: 0.391184
Train Epoch: 1250 [acc: 90%]	Loss: 0.256188
[Trained for 1250 epochs and tested on 5 sets of 2000 images]        Avg Acc: 29.55 +- 1.14 , Avg Loss: 4.54
Train Epoch: 1251 [acc: 95%]	Loss: 0.217475
Train Epoch: 1252 [acc: 88%]	Loss: 0.368473
Train Epoch: 1253 [acc: 88%]	Loss: 0.373994
Train Epoch

Train Epoch: 1403 [acc: 95%]	Loss: 0.191035
Train Epoch: 1404 [acc: 92%]	Loss: 0.217832
Train Epoch: 1405 [acc: 91%]	Loss: 0.198371
Train Epoch: 1406 [acc: 95%]	Loss: 0.195391
Train Epoch: 1407 [acc: 100%]	Loss: 0.060834
Train Epoch: 1408 [acc: 95%]	Loss: 0.171411
Train Epoch: 1409 [acc: 96%]	Loss: 0.147393
Train Epoch: 1410 [acc: 96%]	Loss: 0.126285
Train Epoch: 1411 [acc: 90%]	Loss: 0.260664
Train Epoch: 1412 [acc: 90%]	Loss: 0.282993
Train Epoch: 1413 [acc: 88%]	Loss: 0.284825
Train Epoch: 1414 [acc: 97%]	Loss: 0.134564
Train Epoch: 1415 [acc: 93%]	Loss: 0.235181
Train Epoch: 1416 [acc: 96%]	Loss: 0.114329
Train Epoch: 1417 [acc: 94%]	Loss: 0.286214
Train Epoch: 1418 [acc: 96%]	Loss: 0.104106
Train Epoch: 1419 [acc: 92%]	Loss: 0.284724
Train Epoch: 1420 [acc: 90%]	Loss: 0.328143
Train Epoch: 1421 [acc: 92%]	Loss: 0.213622
Train Epoch: 1422 [acc: 91%]	Loss: 0.206463
Train Epoch: 1423 [acc: 90%]	Loss: 0.349695
Train Epoch: 1424 [acc: 96%]	Loss: 0.175591
Train Epoch: 1425 [acc: 91%]	Lo

Train Epoch: 1575 [acc: 89%]	Loss: 0.277492
[Trained for 1575 epochs and tested on 5 sets of 2000 images]        Avg Acc: 35.06 +- 0.62 , Avg Loss: 4.42
Train Epoch: 1576 [acc: 92%]	Loss: 0.230349
Train Epoch: 1577 [acc: 90%]	Loss: 0.313298
Train Epoch: 1578 [acc: 99%]	Loss: 0.057142
Train Epoch: 1579 [acc: 95%]	Loss: 0.151535
Train Epoch: 1580 [acc: 95%]	Loss: 0.140721
Train Epoch: 1581 [acc: 96%]	Loss: 0.116118
Train Epoch: 1582 [acc: 96%]	Loss: 0.141786
Train Epoch: 1583 [acc: 94%]	Loss: 0.159263
Train Epoch: 1584 [acc: 93%]	Loss: 0.231313
Train Epoch: 1585 [acc: 93%]	Loss: 0.246126
Train Epoch: 1586 [acc: 97%]	Loss: 0.084991
Train Epoch: 1587 [acc: 96%]	Loss: 0.084852
Train Epoch: 1588 [acc: 94%]	Loss: 0.158712
Train Epoch: 1589 [acc: 93%]	Loss: 0.233070
Train Epoch: 1590 [acc: 95%]	Loss: 0.119156
Train Epoch: 1591 [acc: 89%]	Loss: 0.281254
Train Epoch: 1592 [acc: 94%]	Loss: 0.175119
Train Epoch: 1593 [acc: 95%]	Loss: 0.192424
Train Epoch: 1594 [acc: 89%]	Loss: 0.325479
Train Epoch

Train Epoch: 1744 [acc: 92%]	Loss: 0.258450
Train Epoch: 1745 [acc: 95%]	Loss: 0.200334
Train Epoch: 1746 [acc: 92%]	Loss: 0.349224
Train Epoch: 1747 [acc: 90%]	Loss: 0.394925
Train Epoch: 1748 [acc: 94%]	Loss: 0.185571
Train Epoch: 1749 [acc: 95%]	Loss: 0.199728
Train Epoch: 1750 [acc: 96%]	Loss: 0.093285
[Trained for 1750 epochs and tested on 5 sets of 2000 images]        Avg Acc: 32.90 +- 0.67 , Avg Loss: 4.37
Train Epoch: 1751 [acc: 99%]	Loss: 0.065901
Train Epoch: 1752 [acc: 89%]	Loss: 0.335816
Train Epoch: 1753 [acc: 93%]	Loss: 0.194855
Train Epoch: 1754 [acc: 97%]	Loss: 0.118738
Train Epoch: 1755 [acc: 93%]	Loss: 0.198381
Train Epoch: 1756 [acc: 91%]	Loss: 0.310581
Train Epoch: 1757 [acc: 95%]	Loss: 0.143106
Train Epoch: 1758 [acc: 97%]	Loss: 0.146292
Train Epoch: 1759 [acc: 92%]	Loss: 0.199741
Train Epoch: 1760 [acc: 97%]	Loss: 0.086256
Train Epoch: 1761 [acc: 94%]	Loss: 0.187009
Train Epoch: 1762 [acc: 95%]	Loss: 0.152203
Train Epoch: 1763 [acc: 92%]	Loss: 0.270757
Train Epoch

Train Epoch: 1913 [acc: 95%]	Loss: 0.229541
Train Epoch: 1914 [acc: 93%]	Loss: 0.249079
Train Epoch: 1915 [acc: 94%]	Loss: 0.171019
Train Epoch: 1916 [acc: 95%]	Loss: 0.153181
Train Epoch: 1917 [acc: 96%]	Loss: 0.164813
Train Epoch: 1918 [acc: 95%]	Loss: 0.145639
Train Epoch: 1919 [acc: 93%]	Loss: 0.277524
Train Epoch: 1920 [acc: 93%]	Loss: 0.182770
Train Epoch: 1921 [acc: 96%]	Loss: 0.121205
Train Epoch: 1922 [acc: 95%]	Loss: 0.133642
Train Epoch: 1923 [acc: 89%]	Loss: 0.251794
Train Epoch: 1924 [acc: 97%]	Loss: 0.208952
Train Epoch: 1925 [acc: 94%]	Loss: 0.244054
[Trained for 1925 epochs and tested on 5 sets of 2000 images]        Avg Acc: 35.60 +- 0.62 , Avg Loss: 4.03
Train Epoch: 1926 [acc: 93%]	Loss: 0.153874
Train Epoch: 1927 [acc: 99%]	Loss: 0.071766
Train Epoch: 1928 [acc: 93%]	Loss: 0.201775
Train Epoch: 1929 [acc: 93%]	Loss: 0.204459
Train Epoch: 1930 [acc: 94%]	Loss: 0.185598
Train Epoch: 1931 [acc: 94%]	Loss: 0.194224
Train Epoch: 1932 [acc: 94%]	Loss: 0.168477
Train Epoch

In [11]:
dirname = latexTracker[-1][:-2] 

def writeTex(latexTracker,dirname):
    if not os.path.isdir(dirname):
        os.mkdir(dirname)
        
    f= open(os.path.join(dirname,"latexTable.txt"),"w")
    for x in latexTracker:
        f.write(x)
    f.close()

writeTex(latexTracker,dirname)

for x in latexTracker:
    print(x)

WideResNet28&25&16.11&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&50&17.02&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&75&22.75&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&100&21.92&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&125&24.37&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&150&26.47&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&175&27.08&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&200&28.62&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&225&28.76&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&250&27.82&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&275&29.74&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&300&29.38&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&325&26.97&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&350&29.82&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&375&28.88&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&400&31.92&Adam&0.001&10000&Nothing&1620010424\\
WideResNet28&425&32.57&Adam

In [12]:

epochList = [x+1 for x in range(len(trainTracker["meanLoss"]))]

plot(xlist=epochList,ylist=trainTracker["meanLoss"],xlab="Mean Train Loss",
    ylab="Epochs",title="Mean Train Loss over Epochs",
    color="#243A92",label="mean train loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=trainTracker["accuracy"],xlab="Train Accuracy",
    ylab="Epochs",title="Train Accuracy Over Epochs",
    color="#34267E",label="Train Accuracy",savedir=dirname,save=True)

<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [13]:

epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))]

plot(xlist=epochList,ylist=valTracker["meanLoss"],xlab="Epochs",
    ylab="Mean Val Loss",title="Mean Val Loss over Epochs",
    color="#243A92",label="mean val loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["meanAcc"],xlab="Epochs",
    ylab="Val Accuracy",title="Val Accuracy Over Epochs",
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["stdAcc"],xlab="Epochs",
    ylab="Val Accuracy Standard Deviation",title="Val Accuracy Standard Deviation Over Epochs",
    color="#34267E",label="Val Accuracy SD",savedir=dirname,save=True)


valSetEvalCount = VAL_DISPLAY_DIVISOR * EPOCH_NUM * VALIDATION_SET_NUM
epochList = [VAL_DISPLAY_DIVISOR*(x+1) for x in range(len(valTracker["meanLoss"]))\
             for y in range(VALIDATION_SET_NUM)]


plot(xlist=epochList,ylist=valTracker["allLoss"],xlab="Val Set Evaluations",
    ylab="Val Loss",title="Val loss over val set evaluations ({} \
every {} epochs)".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Loss",savedir=dirname,save=True)

plot(xlist=epochList,ylist=valTracker["allAcc"],xlab="Val Set Evaluations",
    ylab="Val Accuracy",title="Val loss over val set evaluations ({} \
every {} epochs) ".format(VALIDATION_SET_NUM,VAL_DISPLAY_DIVISOR),
    color="#34267E",label="Val Accuracy",savedir=dirname,save=True)

<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
<ipython-input-3-b80f051ceb38>:39: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
